In [ ]:
import os
import time
import subprocess
from datetime import datetime, timedelta
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# === CONFIGURACIÓN ===
WATCH_FOLDER = r"C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML"
OUTPUT_FOLDER = r"C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML"
JAR_PATH = r"C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/boda-nokiacmdataparser.jar"

# Ruta completa a java.exe (mi venv no lo detectaba java)
JAVA_EXE = r"C:\Program Files (x86)\Common Files\Oracle\Java\java8path\java.exe"

DIAS_PARA_LIMPIEZA = 7    # elimina dumps con más de 7 días
TIEMPO_ESPERA_COPIA = 10  # segundos entre verificaciones de tamaño

# === FUNCIONES ===
def archivo_completo(path, checks_estables=3, timeout=300):
    """Espera hasta que el archivo deje de crecer (varios checks) o expire timeout."""
    t0 = time.time()
    estables = 0
    last = -1

    while True:
        if time.time() - t0 > timeout:
            raise TimeoutError(f"Timeout esperando copia completa: {path}")

        try:
            size = os.path.getsize(path)
        except FileNotFoundError:
            return False

        if size == last and size > 0:
            estables += 1
            if estables >= checks_estables:
                return True
        else:
            estables = 0

        last = size
        time.sleep(TIEMPO_ESPERA_COPIA)


def procesar_archivo(xml_path):
    nombre = os.path.basename(xml_path)
    print(f"[{datetime.now()}] Procesando: {nombre}")

    try:
        # Validaciones claras antes de ejecutar
        if not os.path.exists(JAVA_EXE):
            raise FileNotFoundError(f"No existe JAVA_EXE: {JAVA_EXE}")
        if not os.path.exists(JAR_PATH):
            raise FileNotFoundError(f"No existe JAR_PATH: {JAR_PATH}")
        if not os.path.exists(xml_path):
            raise FileNotFoundError(f"No existe XML: {xml_path}")

        # Ejecutar JAR
        res = subprocess.run(
            [JAVA_EXE, "-jar", JAR_PATH, "-i", xml_path, "-o", OUTPUT_FOLDER],
            capture_output=True,
            text=True
        )

        if res.returncode != 0:
            print(f"[{datetime.now()}] ❌ Java returncode={res.returncode}")
            if res.stdout:
                print("STDOUT:\n", res.stdout)
            if res.stderr:
                print("STDERR:\n", res.stderr)
            print()
            return

        print(f"[{datetime.now()}] ✅ Conversión completada: {nombre}\n")

    except Exception as e:
        print(f"[{datetime.now()}] ❌ Error al procesar {nombre}: {e}\n")


def procesar_existentes():
    """Procesa archivos XML que ya existen al iniciar."""
    for file in os.listdir(WATCH_FOLDER):
        if file.lower().endswith(".xml"):
            ruta = os.path.join(WATCH_FOLDER, file)
            print(f"[{datetime.now()}] Archivo existente encontrado: {file}")
            if archivo_completo(ruta):
                procesar_archivo(ruta)


def limpiar_archivos_viejos():
    """Elimina dumps viejos del folder."""
    ahora = datetime.now()
    for file in os.listdir(WATCH_FOLDER):
        if file.lower().endswith(".xml"):
            ruta = os.path.join(WATCH_FOLDER, file)
            try:
                mod_time = datetime.fromtimestamp(os.path.getmtime(ruta))
                if ahora - mod_time > timedelta(days=DIAS_PARA_LIMPIEZA):
                    os.remove(ruta)
                    print(f"[{datetime.now()}] 🧹 Eliminado archivo antiguo: {file}")
            except FileNotFoundError:
                pass


# === EVENTO WATCHDOG ===
class DumpHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.lower().endswith(".xml"):
            return

        archivo = event.src_path
        nombre = os.path.basename(archivo)
        print(f"[{datetime.now()}] Nuevo dump detectado: {nombre}")

        if archivo_completo(archivo):
            procesar_archivo(archivo)

    def on_moved(self, event):
        # Por si el sistema crea temp y luego renombra/mueve al final
        if event.is_directory or not event.dest_path.lower().endswith(".xml"):
            return

        archivo = event.dest_path
        nombre = os.path.basename(archivo)
        print(f"[{datetime.now()}] Dump movido/renombrado detectado: {nombre}")

        if archivo_completo(archivo):
            procesar_archivo(archivo)


# === LOOP PRINCIPAL ===
if __name__ == "__main__":
    os.makedirs(WATCH_FOLDER, exist_ok=True)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    print("=== HOT FOLDER NOKIA CM ACTIVO ===")
    print(f"Monitoreando: {WATCH_FOLDER}")
    print(f"Salida CSV:  {OUTPUT_FOLDER}")
    print(f"Limpieza automática: {DIAS_PARA_LIMPIEZA} días\n")

    # Procesar archivos existentes al iniciar
    procesar_existentes()

    # Iniciar monitor
    observer = Observer()
    observer.schedule(DumpHandler(), WATCH_FOLDER, recursive=False)
    observer.start()

    try:
        while True:
            # Si quieres limpieza, descomenta:
            # limpiar_archivos_viejos()
            time.sleep(3600)  # vivo; una vuelta por hora
    except KeyboardInterrupt:
        pass
    finally:
        observer.stop()
        observer.join()


=== HOT FOLDER NOKIA CM ACTIVO ===
Monitoreando: C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML
Salida CSV:  C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML
Limpieza automática: 7 días

[2026-02-18 11:31:43.546138] Archivo existente encontrado: All_Nokia.xml
[2026-02-18 11:32:13.548820] Procesando: All_Nokia.xml
